In [1]:
from posit_decode import decode

In [2]:
import softposit as sp

In [3]:
get_bin = lambda x, n: format(x, "b").zfill(n)
get_hex = lambda x, n: format(x, "x").zfill(n)

In [4]:
bits = 0b01110011

decode(bits, 8, 1)

P<8,1>: 0b01110011
01110011
(-1)**0 * (2**(2**1))**2 * (2**0) * (1 + 3/4) = 28.0
s:                                             0
reg_s:                                             1
reg_len:                                             4
k:                                             2
reg:                                      00001110
exp:                                      00000000
mant:                                      00000011
F = mant_len: 2 -> 2**F = 4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [5]:
# open function signature within jupyter
decode?

Signature: decode(bits, size=2, es=0) -> posit.Posit
Docstring:
Posit decoder.

Break down P<size, es> in its components (sign, regime, exponent, mantissa).

Prameters:
bits (unsigned): sequence of bits representing the posit
size (unsigned): length of posit
es (unsigned): exponent field size.

Returns:
Posit object
File:      ~/Dropbox/UNI/MASTER/thesis/msc-thesis/RISCV-PPU/ppu/scripts/posit_decode.py
Type:      function


In [6]:
decode(bits=bits, size=8, es=0)

P<8,0>: 0b01110011
01110011
(-1)**0 * (2**(2**0))**2 * (2**0) * (1 + 3/8) = 5.5
s:                                             0
reg_s:                                             1
reg_len:                                             4
k:                                             2
reg:                                      00001110
mant:                                      00000011
F = mant_len: 3 -> 2**F = 8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [7]:
p1 = decode(0b01111100000110101100, 20, 0)

p1

P<20,0>: 0b01111100000110101100
01111100000110101100
(-1)**0 * (2**(2**0))**4 * (2**0) * (1 + 428/8192) = 16.8359375
s:                                             0
reg_s:                                             1
reg_len:                                             6
k:                                             4
reg:                          00000000000000111110
mant:                          00000000000110101100
F = mant_len: 13 -> 2**F = 8192
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [8]:
p2 = decode(0b11111100010110100100, 20, 0)
p2

P<20,0>: 0b11111100010110100100
10000011101001011100
(-1)**1 * (2**(2**0))**-5 * (2**0) * (1 + 6748/8192) = -0.0569915771484375
s:                                             1
reg_s:                                             0
reg_len:                                             6
k:                                            -5
reg:                          00000000000000000001
mant:                          00000001101001011100
F = mant_len: 13 -> 2**F = 8192
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [9]:
from mul import mul

In [10]:
mul(p1, p2)

                    10000110101100000000 x
                    11101001011100000000 =
------------------------------------------
0111101011010001000111010000000000000000

mant_carry = 0
k + exp + mant_carry = -1 + 0 + 0
k + exp + mant_carry = -1 + 0 + 0


P<20,0>: 0b11000010100101111000
10111101011010001000
(-1)**1 * (2**(2**0))**-1 * (2**0) * (1 + 120456/131072) = -0.959503173828125
s:                                             1
reg_s:                                             0
reg_len:                                             2
k:                                            -1
reg:                          00000000000000000001
mant:                          00011101011010001000
F = mant_len: 17 -> 2**F = 131072
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [27]:
import random

NUM_RANDOM_TEST_CASES = 100

N = 16

list_a = random.sample(range(0, 2 ** N - 1), min(NUM_RANDOM_TEST_CASES, 2 ** N - 1))
list_b = random.sample(range(0, 2 ** N - 1), min(NUM_RANDOM_TEST_CASES, 2 ** N - 1))

errors = []

for a,b in zip(list_a, list_b):
    if N == 8:
        softposit = ( sp.posit8(bits=a) * sp.posit8(bits=b) )
        ES = 0
    elif N == 16:
        softposit = ( sp.posit16(bits=a) * sp.posit16(bits=b) )
        ES = 1
    
    mine = mul(decode(a, N, ES), decode(b, N, ES)).to_real() 
    if mine == softposit:
        pass
    else:
        errors.append((a,b,mine,softposit))

                1110110110100000 x
                1101110010110000 =
----------------------------------
11001100110110001101111000000000

mant_carry = 1
k + exp + mant_carry = -1 + 1 + 1
k + exp + mant_carry = -1 + 2 + 0
                1011010000000000 x
                1101110101110000 =
----------------------------------
10011011101100101100000000000000

mant_carry = 1
k + exp + mant_carry = -3 + 1 + 1
k + exp + mant_carry = -3 + 2 + 0
                1111010011000000 x
                1101010010101000 =
----------------------------------
11001011010011111001111000000000

mant_carry = 1
k + exp + mant_carry = 0 + 0 + 1
k + exp + mant_carry = 1 + 0 + 0
                1100011001100000 x
                1111110011000000 =
----------------------------------
11000011110110110100100000000000

mant_carry = 1
k + exp + mant_carry = 2 + 0 + 1
k + exp + mant_carry = 3 + 0 + 0
                1111100100011000 x
                1000101111100000 =
----------------------------------
10001000000

In [28]:
errors

[(41548, 54378, 0.40008544921875, 1.600341796875),
 (55680, 7639, -0.0189971923828125, -0.076019287109375),
 (20120, 46443, -6.3515625, -3.1767578125),
 (18636, 35866, -97.875, -48.96875),
 (41181, 56964, 0.26580810546875, 1.063232421875),
 (8949, 42530, -1.915283203125, -0.957763671875),
 (47281, 33670, 1096.0, 550.0),
 (34804, 28747, -1120.0, -1128.0),
 (1311, 26878, 0.109619140625, 0.0548095703125),
 (39645, 1405, -0.0894775390625, -0.04473876953125),
 (26197, 30329, 92.75, 371.0),
 (23121, 22479, 4.89453125, 9.7890625),
 (54440, 38621, 7.806640625, 3.90380859375),
 (11421, 15655, 0.101806640625, 0.40728759765625),
 (44334, 5739, -0.06622314453125, -0.2650146484375),
 (20532, 50268, -0.43731689453125, -1.74951171875),
 (17559, 15856, 0.30096435546875, 1.203857421875),
 (61398, 46046, 0.11212158203125, 0.112152099609375),
 (20186, 55104, -1.4912109375, -0.7457275390625),
 (63169, 31688, -1.23974609375, -4.958984375),
 (27543, 15676, 5.29296875, 10.58984375),
 (49176, 12194, -0.123199

In [29]:
print("{} % are wrong".format(len(errors)/len(list_a) * 100))

96.0 % are wrong
